In [ ]:
# pip install sentence_transformers
# pip install langchain
# pip install pypdf
# pip install typing-extensions
# pip install chromadb
# pip install unstructured

In [7]:
from langchain.llms import LlamaCpp

MODEL_PATH = "../models/dolphin-2.5-mixtral-8x7b.Q4_K_M.gguf"
MODEL_PATH = "/home/flash/models/dolphin-2.5-mixtral-8x7b.Q4_K_M.gguf"
# 38min
# 40s
llm = LlamaCpp(
    model_path=MODEL_PATH,
    n_ctx=1024 * 32,
    n_gpu_layers=40,
    n_threads=15,
    n_batch=512,
    f16_kv=True,
    #callback_manager=callback_manager,
    verbose=True,
    temperature=0.2,
    max_tokens=1024
)

............................................................................
llama_new_context_with_model: n_ctx      = 32768
llama_new_context_with_model: freq_base  = 10000.0
llama_new_context_with_model: freq_scale = 1
llama_new_context_with_model: KV self size  = 4096.00 MiB, K (f16): 2048.00 MiB, V (f16): 2048.00 MiB
llama_build_graph: non-view tensors processed: 1124/1124
llama_new_context_with_model: compute buffer total size = 2167.22 MiB
llama_new_context_with_model: VRAM scratch buffer: 2164.04 MiB
llama_new_context_with_model: total VRAM used: 27309.60 MiB (model: 25145.56 MiB, context: 2164.04 MiB)
AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


In [27]:
def ask(user_input):
   system_prompt = """I'm a QA expert, I'll respond only to answers I'm confident in, and for ones I'm not sure about, I'll simply say 'I don't know'."""
   prompt_template = """<|im_start|>system
{system_prompt}<|im_end|>
<|im_start|>user
{user_prompt}<|im_end|>
<|im_start|>assistant
"""
   prompt = prompt_template.format(system_prompt=system_prompt, user_prompt=user_input)
   resp = llm(prompt,  
      max_tokens=2048,
      temperature=0.7,
      top_p=0.1,
      repeat_penalty=1.1)
   return resp

In [28]:
answer = ask("What is your name?")
answer

Llama.generate: prefix-match hit

llama_print_timings:        load time =    4787.84 ms
llama_print_timings:      sample time =       2.50 ms /     7 runs   (    0.36 ms per token,  2797.76 tokens per second)
llama_print_timings: prompt eval time =    1015.09 ms /    62 tokens (   16.37 ms per token,    61.08 tokens per second)
llama_print_timings:        eval time =     276.69 ms /     6 runs   (   46.11 ms per token,    21.69 tokens per second)
llama_print_timings:       total time =    1327.97 ms


" I don't know."

In [13]:
def read_file(file):
   with open(file, 'r', encoding='utf-8') as f:
      return f.read()

In [2]:
from py_standard.langchain_lit import split_documents, convert_docs_to_splits, LlmEmbedding, load_markdown_documents
from py_standard.pdf_utils import load_pdf_documents_from_directory
from langchain.vectorstores import Chroma

EMB_MODEL = "bge-base-en"
#EMB_MODEL = "bge-large-zh-v1.5"
EMB_MODEL = "UAE-Large-V1"

def load_all_pdfs(path):
   txts = load_markdown_documents('./mds')
   pdfs = load_pdf_documents_from_directory(path)
   all_docs = txts + pdfs
   docs = split_documents(all_docs, 500)
   return docs
   
def load_vectorstore(docs):
   llm_embedding = LlmEmbedding(f"../models/{EMB_MODEL}")
   vectorstore = Chroma.from_documents(documents=docs,
                                        embedding=llm_embedding.embedding,
                                        collection_metadata={"hnsw:space": "cosine",
                                                             "hnsw:search_ef" : 100})
   return vectorstore
   # retriever = vectorstore.as_retriever(
   #   search_kwargs={'k': 3, 'fetch_k': 50}
   # )
   # return retriever


In [3]:
docs = load_all_pdfs('./pdfs')
vectorstore = load_vectorstore(docs)
len(docs)

No sentence-transformers model found with name ../models/UAE-Large-V1. Creating a new one with MEAN pooling.


2477

In [4]:
def search_docs(user_query):
   docs = vectorstore.similarity_search_with_score(user_query, k=5)
   return docs

In [5]:
question = "What is your name?"

splits = []
for doc, score in search_docs(question):
   item = {
      'page_content': doc.page_content,
      'score': score,
      'source': doc.metadata['source']
   }
   splits.append(item)
   
   #print(page_content)  
#splits = convert_docs_to_splits(docs)
for split in splits:
   print(f"{split['score']=} {split['source']=}")

split['score']=0.29337143898010254 split['source']='mds/test.md'
split['score']=0.38078171014785767 split['source']='pdfs/Five-CountBaccarat-Book.pdf'
split['score']=0.38078171014785767 split['source']='pdfs/Silverthorne Publications, Inc..pdf'
split['score']=0.4123804569244385 split['source']='pdfs/pdfcoffee.com_targetbaccarat-bookpdf-pdf-free.pdf'
split['score']=0.4290034770965576 split['source']='pdfs/The Ultimate Baccarat Strategy.pdf'


In [10]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains.question_answering import load_qa_chain

def create_qa_chain_prompt_template():
    system_prompt = """You are a helpful assistant, you will use the provided context to answer user questions.
Read the given context before answering questions and think step by step. 
If you can not answer a user question based on the provided context, inform the user. 
Do not use any other information for answering user."""

    prompt_template = """Instruction: Answer the question based on following context. 
If the answer cannot be found from the following context, respond with 'I don't know.'.
Here is context to help:

{context}
    
### QUESTION:
{question}
    """
    
    prompt_template1 = system_prompt + """
Here is context to help:

{context}
    
### QUESTION:
{question}
    """

    qa_chain_prompt = PromptTemplate(
        input_variables=["context", "question"],
        template=prompt_template
    )
    return qa_chain_prompt

# Chain
qa_chain = load_qa_chain(
    llm,
    chain_type="stuff",
    prompt=create_qa_chain_prompt_template()
)

# Run
def ask_qa(user_question):
    all_docs = search_docs(user_question)
    docs = []
    for doc, score in all_docs:
        docs.append(doc)
    result = qa_chain({
        "input_documents": docs,
        "question": user_question
        },
        return_only_outputs=True
    )
    return result['output_text']

In [11]:
ask_qa("What is your name?")


llama_print_timings:        load time =    2087.39 ms
llama_print_timings:      sample time =       5.21 ms /    14 runs   (    0.37 ms per token,  2687.14 tokens per second)
llama_print_timings: prompt eval time =    2086.66 ms /   190 tokens (   10.98 ms per token,    91.05 tokens per second)
llama_print_timings:        eval time =     660.87 ms /    13 runs   (   50.84 ms per token,    19.67 tokens per second)
llama_print_timings:       total time =    2791.80 ms


'\nA: Your name is Astro. Flash created me.'

In [12]:
ask_qa("What is Five-Count Baccarat?")

Llama.generate: prefix-match hit

llama_print_timings:        load time =    2087.39 ms
llama_print_timings:      sample time =      12.99 ms /    37 runs   (    0.35 ms per token,  2849.22 tokens per second)
llama_print_timings: prompt eval time =    2170.95 ms /   214 tokens (   10.14 ms per token,    98.57 tokens per second)
llama_print_timings:        eval time =    2029.62 ms /    36 runs   (   56.38 ms per token,    17.74 tokens per second)
llama_print_timings:       total time =    4288.49 ms


'\n### ANSWER:\n Five-Count Baccarat is a variant of the card game baccarat, which has its own set of rules and strategies.'

In [13]:
ask_qa("What is Flat betting won?")

Llama.generate: prefix-match hit

llama_print_timings:        load time =    2087.39 ms
llama_print_timings:      sample time =       6.20 ms /    17 runs   (    0.36 ms per token,  2741.05 tokens per second)
llama_print_timings: prompt eval time =    4795.85 ms /   501 tokens (    9.57 ms per token,   104.47 tokens per second)
llama_print_timings:        eval time =    1000.22 ms /    16 runs   (   62.51 ms per token,    16.00 tokens per second)
llama_print_timings:       total time =    5833.08 ms


'\n### ANSWER:\n About 39% of its games.'

In [14]:
ask_qa("What is Flat betting?")

Llama.generate: prefix-match hit

llama_print_timings:        load time =    2087.39 ms
llama_print_timings:      sample time =       9.29 ms /    26 runs   (    0.36 ms per token,  2799.31 tokens per second)
llama_print_timings: prompt eval time =    5189.94 ms /   531 tokens (    9.77 ms per token,   102.31 tokens per second)
llama_print_timings:        eval time =    1544.06 ms /    25 runs   (   61.76 ms per token,    16.19 tokens per second)
llama_print_timings:       total time =    6794.89 ms


'\n### ANSWER:\n Flat betting is when a beginner makes the same size bet over and over again.'

In [15]:
ask_qa("How do you engage in Flat betting?")

Llama.generate: prefix-match hit

llama_print_timings:        load time =    2087.39 ms
llama_print_timings:      sample time =      16.98 ms /    46 runs   (    0.37 ms per token,  2708.75 tokens per second)
llama_print_timings: prompt eval time =    5305.62 ms /   530 tokens (   10.01 ms per token,    99.89 tokens per second)
llama_print_timings:        eval time =    2949.11 ms /    45 runs   (   65.54 ms per token,    15.26 tokens per second)
llama_print_timings:       total time =    8404.98 ms


'\n### ANSWER:\n To engage in flat betting, you simply make the same size bet over and over again. This approach offers the greatest likelihood of keeping your bankroll, but the poorest chance of winning.'

In [16]:
ask_qa("What's the strategy for playing a coin-toss game?")

Llama.generate: prefix-match hit

llama_print_timings:        load time =    2087.39 ms
llama_print_timings:      sample time =      23.25 ms /    60 runs   (    0.39 ms per token,  2580.42 tokens per second)
llama_print_timings: prompt eval time =    6686.94 ms /   680 tokens (    9.83 ms per token,   101.69 tokens per second)
llama_print_timings:        eval time =    4330.64 ms /    59 runs   (   73.40 ms per token,    13.62 tokens per second)
llama_print_timings:       total time =   11183.29 ms


'\n### ANSWER:\n The strategy for playing a coin-toss game is to always wager heads, never bet tie, and use a betting system that varies the size of your bankroll and the number of games played. Each game consists of 100 bets.'

In [17]:
ask_qa("In betting strategies, what percentage of games does flat betting win?")

Llama.generate: prefix-match hit

llama_print_timings:        load time =    2087.39 ms
llama_print_timings:      sample time =       8.45 ms /    23 runs   (    0.37 ms per token,  2722.54 tokens per second)
llama_print_timings: prompt eval time =    6088.41 ms /   612 tokens (    9.95 ms per token,   100.52 tokens per second)
llama_print_timings:        eval time =    1409.57 ms /    22 runs   (   64.07 ms per token,    15.61 tokens per second)
llama_print_timings:       total time =    7559.31 ms


'\n### ANSWER:\n Flat betting wins 38.70% of the games.'

In [18]:
ask_qa("How to Starter bets?")

Llama.generate: prefix-match hit

llama_print_timings:        load time =    2087.39 ms
llama_print_timings:      sample time =      33.84 ms /    91 runs   (    0.37 ms per token,  2689.28 tokens per second)
llama_print_timings: prompt eval time =    5811.16 ms /   602 tokens (    9.65 ms per token,   103.59 tokens per second)
llama_print_timings:        eval time =    6309.73 ms /    90 runs   (   70.11 ms per token,    14.26 tokens per second)
llama_print_timings:       total time =   12327.57 ms


'\n### ANSWER:\n A starter bet is a neutral bet made only to start a game. You will need to make this bet whenever the house requires a wager in order to start a game, and you are the only player at the table. You will encounter this situation with online baccarat games where you are playing a computer-controlled game which will only deal a round of play if you have a bet placed.'

In [19]:
ask_qa("Can you explain the concept of 'Starter bets' as presented in the article?")

Llama.generate: prefix-match hit

llama_print_timings:        load time =    2087.39 ms
llama_print_timings:      sample time =      38.81 ms /   102 runs   (    0.38 ms per token,  2628.19 tokens per second)
llama_print_timings: prompt eval time =    6546.41 ms /   669 tokens (    9.79 ms per token,   102.19 tokens per second)
llama_print_timings:        eval time =    7277.35 ms /   101 runs   (   72.05 ms per token,    13.88 tokens per second)
llama_print_timings:       total time =   14055.21 ms


'\n### ANSWER:\n Starter bets are wagers made to start a game when you don’t have two previous decisions to observe.  You will need to make these bets whenever the house requires a wager in order to start a game and you are the only player at the table.   You will encounter this situation with online baccarat games where you are playing a computer-controlled game which will only deal a round of play if you have a bet placed.'

In [20]:
ask_qa("What is Attack Bets?")

Llama.generate: prefix-match hit

llama_print_timings:        load time =    2087.39 ms
llama_print_timings:      sample time =      21.87 ms /    62 runs   (    0.35 ms per token,  2834.93 tokens per second)
llama_print_timings: prompt eval time =    4714.74 ms /   506 tokens (    9.32 ms per token,   107.32 tokens per second)
llama_print_timings:        eval time =    3886.74 ms /    61 runs   (   63.72 ms per token,    15.69 tokens per second)
llama_print_timings:       total time =    8738.27 ms


'\n### ANSWER:\n Attack Bets are a type of bet that follows the same playing rules for any level of play. These bets are always started at the lowest level Attack Bet and can trigger a move to Recovery Betting if the highest level Attack Bet is lost.'

In [21]:
ask_qa("What are the Two Baccarat Decisions You Have to Make?")

Llama.generate: prefix-match hit

llama_print_timings:        load time =    2087.39 ms
llama_print_timings:      sample time =      12.82 ms /    35 runs   (    0.37 ms per token,  2730.32 tokens per second)


'\n### ANSWER:\n The two baccarat decisions you have to make are where do you place each bet and how much do you bet each time.'

llama_print_timings: prompt eval time =    5967.32 ms /   600 tokens (    9.95 ms per token,   100.55 tokens per second)
llama_print_timings:        eval time =    2165.01 ms /    34 runs   (   63.68 ms per token,    15.70 tokens per second)
llama_print_timings:       total time =    8210.66 ms


####

In [31]:
context = read_file('./data/baccarat-book.md')

In [26]:
def split_string(text, length):
    return [text[i:i+length] for i in range(0, len(text), length)]

In [32]:
context_splits = split_string(context, 6000)

In [33]:
qa_prompt_template = """Instruction: Answer the question based on following context. 
If the answer cannot be found from the following context, respond with 'I don't know.'.
Here is context to help:

{context}

### USER QUESTION:
{question} 
"""

prompt = qa_prompt_template.format(context=context_splits[0],question="What is baccarat?")
answer = ask(prompt)
answer

Llama.generate: prefix-match hit

llama_print_timings:        load time =    1408.84 ms
llama_print_timings:      sample time =      93.50 ms /   256 runs   (    0.37 ms per token,  2737.97 tokens per second)
llama_print_timings: prompt eval time =   21537.70 ms /  1750 tokens (   12.31 ms per token,    81.25 tokens per second)
llama_print_timings:        eval time =   30391.78 ms /   255 runs   (  119.18 ms per token,     8.39 tokens per second)
llama_print_timings:       total time =   52509.51 ms


" Baccarat is a popular card game played in casinos around the world. The objective of the game is for the player to predict whether the value of their hand will be closer to or equal to the value of the banker's hand. The game is usually played with eight decks of cards, and the value of each card is determined as follows: an ace counts as 1 point, a two through nine count as the face value of the card (for example, a seven counts as 7 points), and a ten or face card (king, queen) counts as 0 points. The total value of each player's hand is then calculated by adding together the values of all of the cards in that player's hand. For example, if a player was holding a two and an ace, the total value of their hand would be calculated as follows: 2 (two) + 1 (ace) = 3 points. Similarly, if a player was holding a seven and an eight, the total value of their hand would be calculated as follows: 7 (seven) + 8 (eight) = 15 points. However, since the game is usually played with eight decks of 

In [34]:
def ask_qa(user_input):
   prompt = qa_prompt_template.format(context=context_splits[0],question=user_input)
   return ask(prompt)

In [35]:
answer = ask_qa("What is Contra-d'Alembert?")
answer

Llama.generate: prefix-match hit

llama_print_timings:        load time =    1408.84 ms
llama_print_timings:      sample time =      44.83 ms /   120 runs   (    0.37 ms per token,  2676.90 tokens per second)
llama_print_timings: prompt eval time =    1164.92 ms /    18 tokens (   64.72 ms per token,    15.45 tokens per second)
llama_print_timings:        eval time =   13307.87 ms /   119 runs   (  111.83 ms per token,     8.94 tokens per second)
llama_print_timings:       total time =   14733.31 ms


" Contra-d'Alembert is a betting system in the game of roulette. The name itself is derived from French mathematician Jean le Rond d'Alembert. \n\nThe concept behind this betting system is contrary to that of the famous martingale betting system. Unlike the martingale system, which involves increasing your bets after losing rounds, Contra-d'Alembert requires you to decrease your bets after losing rounds and increase them after winning ones. This approach aims at minimizing losses while maximizing profits."

In [45]:
answer = ask_qa('What is "Tracker Betting"?')
answer

Llama.generate: prefix-match hit

llama_print_timings:        load time =    1408.84 ms
llama_print_timings:      sample time =      31.19 ms /    84 runs   (    0.37 ms per token,  2693.26 tokens per second)
llama_print_timings: prompt eval time =    1074.32 ms /    15 tokens (   71.62 ms per token,    13.96 tokens per second)
llama_print_timings:        eval time =    8756.51 ms /    83 runs   (  105.50 ms per token,     9.48 tokens per second)
llama_print_timings:       total time =   10017.74 ms


' Tracker Betting is a betting strategy used in baccarat play. It involves tracking the results of each round of play to determine where to place bets for subsequent rounds of play. The goal of Tracker Betting is to identify patterns in the outcomes of each round of play and then use this information to make informed decisions about where to place bets in order to maximize profits and minimize losses.'

In [44]:
answer = ask_qa('Generate many questions(Q:) and answers(A:), and the answers must be contained in the above content.')
answer

Llama.generate: prefix-match hit

llama_print_timings:        load time =    1408.84 ms
llama_print_timings:      sample time =     745.15 ms /  2048 runs   (    0.36 ms per token,  2748.45 tokens per second)
llama_print_timings: prompt eval time =    1044.97 ms /    32 tokens (   32.66 ms per token,    30.62 tokens per second)
llama_print_timings:        eval time =  334022.37 ms /  2047 runs   (  163.18 ms per token,     6.13 tokens per second)
llama_print_timings:       total time =  342075.28 ms


' A: Sure, here are some questions and answers based on the context provided:\n\nQ1: What is Target Baccarat?\nA1: Target Baccarat is a professional-level baccarat betting strategy that has never lost in over 10,000 documented games played by real players.\nQ2: How does Target Baccarat work?\nA2: Target Baccarat uses a combination of Target Betting and Tracker Betting to determine where to bet and how much to bet. The strategy also includes a Unique Bet Timing System that is used to pick the optimal time to place each bet.\n\nQ3: What are some key features of Target Baccarat?\nA3: Some key features of Target Baccarat include its ability to consistently produce explosive profits, its use of a combination of Target Betting and Tracker Betting to determine where to bet and how much to bet, its inclusion of a Unique Bet Timing System that is used to pick the optimal time to place each bet, and its overall ease of use and ability to be quickly learned and easily applied by both new and expe

In [43]:
answer = ask_qa('summarize the context')
answer

Llama.generate: prefix-match hit

llama_print_timings:        load time =    1408.84 ms
llama_print_timings:      sample time =      33.60 ms /    89 runs   (    0.38 ms per token,  2648.49 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =    9647.51 ms /    89 runs   (  108.40 ms per token,     9.23 tokens per second)
llama_print_timings:       total time =    9841.62 ms


' Target Baccarat is a professional-level baccarat strategy that has never lost. It uses Target Betting, a system borrowed from stock trading, to set up winning bets. The strategy also determines the size of bets using a Unique Betting Strategy. Overall, Target Baccarat provides an easy-to-learn and use professional-level baccarat strategy that has never lost.'